#####Renomeando Colunas withColumnRenamed e Alias

In [0]:
df = spark.read.json('/Volumes/workspace/default/arquivos-cursoudemy/Anac/V_OCORRENCIA_AMPLA.json')
display(df)

In [0]:
#Sintaxe
#df = df.withColumnRenamed("coluna_antiga","coluna_nova")

In [0]:
print(df.columns)

In [0]:
renomeado = df.withColumnRenamed('Aerodromo_de_Destino','Destino')\
            .withColumnRenamed('Aerodromo_de_Origem','Origem')\
            .withColumnRenamed('Danos_a_Aeronave','Danos')\
            .withColumnRenamed('UF', 'Estado')
display(renomeado)

In [0]:
print(renomeado.columns)

In [0]:
Teste = df.select(df['Aerodromo_de_Destino'].alias('Destino'),
                  df['Aerodromo_de_Origem'].alias('Origem'),
                  df['Danos_a_Aeronave'].alias('Danos'),
                  df['UF'].alias('Estado'),
                  )
display(Teste)

#####Conversão de Dados método cast + withColumn 

In [0]:
#Resumo
#A função cast no PySpark aceita vários tipos de dados para a conversão
"""
string ou varchar: Texto.
integer ou int:    Numero inteiro.
long:              Numero inteiro longo.
double:            precisão dupla.
float:             precisão simples.
decimal:           decimal.
timestamp:         Data e hora.
date:              Somente a Data.
boolean:           booleana (True e False) 0=False e 1=True.

"""

In [0]:
#Curiosidades
"""
Qual diferença entre esses que trazem numeros decimais no geral
double:            precisão dupla.
float:             precisão simples.
decimal:           decimal.

----- numeros com longas casas decimais 
float (Precisão Simples 32 bits na memória):
Mínimo: 1.40129846432 x 10^-45
Máximo: 3.40282346639 x 10^38

double (Precisão Dupla 64 bits na memória):
Mínimo: 4.94065645841 x 10^-324
Máximo: 1.79769313486 x 10^308

decimal : pode escolher quantas casas decimais aparece no resultado 
O decimal usa uma representação decimal, o que permite maior precisão e evita muitos dos problemas de arredondamento associados ao float

Decimal ou Float são mais ultilizados

"""

In [0]:
#Carga Df sem .option("inferSchema", "true") sendo assim virá tudo como string, vamor ver na prática 
orders_df = spark.read.\
    format("csv")\
    .option("header", "true")\
    .load("/Volumes/workspace/default/arquivos-cursoudemy/Bike/orders.csv")
display(orders_df)

In [0]:
orders_df.printSchema()

In [0]:
#Converter dados com withColumn
orders_df = orders_df.withColumn('order_id',orders_df['order_id'].cast('int') )
orders_df = orders_df.withColumn('customer_id',orders_df['customer_id'].cast('integer') ) 
orders_df = orders_df.withColumn('order_date',orders_df['order_date'].cast('date'))
orders_df = orders_df.withColumn('required_date',orders_df['required_date'].cast('date'))
display(orders_df)

In [0]:
#Carga Df sem .option("inferSchema", "true") sendo assim virá tudo como string, vamor ver na prática 
orders_df2 = spark.read.\
    format("csv")\
    .option("header", "true")\
    .load("/Volumes/workspace/default/arquivos-cursoudemy/Bike/orders.csv")
display(orders_df2)

In [0]:
orders_df2 = orders_df2.select(
                orders_df2['order_id'].cast("integer"),
                orders_df2['order_date'].cast("date"),
                orders_df2['store_id'].cast("int"),
                orders_df2['staff_id'].cast("int").alias("IdLoja"),
                orders_df2['order_status'].cast("int").alias("Status_Entrega"),
)
display(orders_df2)

#####Conversões númericas Avançadas

In [0]:
Items = spark.read\
    .format("csv")\
    .option("header", "true")\
    .option("inferSchema", "true")\
    .load("/Volumes/workspace/default/arquivos-cursoudemy/Bike/order_items.csv")
display(Items)

In [0]:
print(Items.columns)

In [0]:
#formato double
fracionado = Items.withColumn('Fracao',Items['list_price']/2.03)
display(fracionado)

In [0]:
#criando novas colunas ja convertidas
fracionado2 = fracionado\
                        .withColumn('double',fracionado['Fracao'])\
                        .withColumn('Inteiro',fracionado['Fracao'].cast('int'))\
                        .withColumn('Float',fracionado['Fracao'].cast('float'))\
                        .withColumn('Decimal',fracionado['Fracao'].cast('decimal(10,2)'))
display(fracionado2)

In [0]:
#Observação
"""
Obs : 'decimal(10,2)'  
o valor decimal terá um máximo de 10 dígitos no total (incluindo os dígitos à esquerda e à direita do ponto decimal) e 2 dígitos à direita do ponto decimal.
"""

In [0]:
#Substituindo o double por decimal no Df
display(fracionado)

In [0]:
fracionado = fracionado.withColumn('Fracao',fracionado['Fracao'].cast('decimal(10,2)'))
display(fracionado)

#####Arredondamento de Numeros

In [0]:
#Caso a maquina reset, rodar a parte acima até gerar o Df abaixo (fracionado2)
display(fracionado2)

In [0]:
#Aproveitar  script fracionado2 para Treinar 
"""
floor (arredondamento para baixo)
ceil  (arredondamento para cima)
round (arredondamento Automatico)
"""

from pyspark.sql.functions import floor, ceil , round
Arred = fracionado2\
    .withColumn('ArredAcima', ceil(fracionado2.double))\
    .withColumn('ArredAbaixo',floor(fracionado2.double))\
    .withColumn('ArredAutomatico',round(fracionado2.double))
display(Arred)


#####Trabalhando com Datas

In [0]:
orders_df = spark.read\
    .format("csv")\
    .option("header", "true")\
    .option("inferSchema", "true")\
    .load("/Volumes/workspace/default/arquivos-cursoudemy/Bike/orders.csv")
display(orders_df)

In [0]:
orders_df.printSchema()

In [0]:
#Extraindo partes das datas através de um campo
from pyspark.sql.functions import year,month,dayofmonth

orders_df = orders_df\
    .withColumn('ano_pedido',year(orders_df['order_date']))\
    .withColumn('mes_pedido',month(orders_df['order_date']))\
    .withColumn('dia_pedido',dayofmonth(orders_df['order_date']))
display(orders_df)


In [0]:
#usando a classe datetime, expr(permite calculos com datas)
from pyspark.sql.functions import expr
from datetime import datetime
orders_df2 = orders_df\
                     .withColumn("ano_pedido", expr("year(order_date)")) \
                     .withColumn("mes_pedido", expr("month(order_date)")) \
                     .withColumn("dia_pedido", expr("day(order_date)"))
display(orders_df2)

In [0]:
#Simulando manter registro de quando os dados foram carregados (Muito comum no dia a dia para rastreio se os dados foram atualizados ou nao )

#formato brasil -3h
#   .withColumn("data_hora_br", expr("from_utc_timestamp(current_timestamp(), 'America/Sao_Paulo')"))

from pyspark.sql.functions import current_date,current_timestamp ,expr

orders_df = orders_df \
    .withColumn("data_atual", current_date()) \
    .withColumn("data_hora_atual", current_timestamp()) \
    .withColumn("data_hora_br", expr("from_utc_timestamp(current_timestamp(), 'America/Sao_Paulo')"))

display(orders_df)


In [0]:
"""
referencias 
https://docs.databricks.com/en/sql/language-manual/functions/from_utc_timestamp.html
https://spark.apache.org/docs/3.1.2/api/python/reference/api/pyspark.sql.functions.from_utc_timestamp.html
"""

In [0]:
#Extraindo horas minutos e Segundos 
from pyspark.sql.functions import year, month, dayofmonth, hour, minute, second
orders_df = orders_df\
                     .withColumn("hora_carga", hour("data_hora_br")) \
                     .withColumn("minuto_carga", minute("data_hora_br")) \
                     .withColumn("segundo_carga", second("data_hora_br"))

display(orders_df)



##### Comandos e Formatos de Datas avançado 

In [0]:
orders_df = spark.read\
    .format("csv")\
    .option("header", "true")\
    .option("inferSchema", "true")\
    .load("/Volumes/workspace/default/arquivos-cursoudemy/Bike/orders.csv")
display(orders_df)

In [0]:
#Formato PT-Br

from pyspark.sql.functions import date_format
orders_df = orders_df.withColumn("data_formatada",date_format(orders_df['order_date'],'dd/MM/yyyy'))
display(orders_df)

In [0]:
#Exemplos outros formatos
from pyspark.sql.functions import date_format 

orders_df = orders_df\
        .withColumn("Exemplo1", date_format(orders_df['order_date'],"yyyy MM dd"))\
        .withColumn("Exemplo2", date_format(orders_df['order_date'],"MM/dd/yyyy HH:mm"))\
        .withColumn("Exemplo3", date_format(orders_df['order_date'],"MM-yyyy"))\
        .withColumn("Exemplo4", date_format(orders_df['order_date'],"yyyy MMM dd"))\
        .withColumn("Exemplo5", date_format(orders_df['order_date'],"yyyy MMMM dd E"))\
        .withColumn("Exemplo6", date_format(orders_df['order_date'],"MMM"))\
        .withColumn("Exemplo7", date_format(orders_df['order_date'],"MMM-yyyy"))   

display(orders_df)


##### Traduçoes para portugues


In [0]:
#conversao para portugues método when 
from pyspark.sql.functions import when
orders_df2 = orders_df.withColumn("Exemplo6",
                                    when(orders_df['Exemplo6'] =="Jan","Jan")
                                   .when(orders_df['Exemplo6'] =="Feb","Fev")
                                   .when(orders_df['Exemplo6'] =="Mar","Mar")
                                   .when(orders_df['Exemplo6'] =="Apr","Abr")
                                   .when(orders_df['Exemplo6'] =="May","Mai")
                                   .when(orders_df['Exemplo6'] =="Jun","Jun")
                                   .when(orders_df['Exemplo6'] =="Jul","Jul")
                                   .when(orders_df['Exemplo6'] =="Aug","Ago")
                                   .when(orders_df['Exemplo6'] =="Sep","Set")
                                   .when(orders_df['Exemplo6'] =="Oct","Out")
                                   .when(orders_df['Exemplo6'] =="Nov","Nov")
                                   .when(orders_df['Exemplo6'] =="Dec","Dez")
                                   .otherwise(orders_df['Exemplo6']))

display(orders_df2)


In [0]:
#Colinha
"""
MMM >mes abrev 
"Jan Feb Mar Apr May Jun Jul Aug Sep Oct Nov Dec",
"Jan Fev Mar Abr Mai Jun Jul Ago Set Out Nov Dez"

MMMM > mes completo
"January February March April May June July August September October November December",
"Janeiro Fevereiro Março Abril Maio Junho Julho Agosto Setembro Outubro Novembro Dezembro"


E > dia da semana 
"Sun Mon Tue Wed Thu Fri Sat", 
"Dom Seg Ter Qua Qui Sex Sáb"

EEEE > dia da semana completo
"Sunday Monday Tuesday Wednesday Thursday Friday Saturday", 
"Domingo Segunda-feira Terça-feira Quarta-feira Quinta-feira Sexta-feira Sábado"


"""

#####Diferença entre Datas Adição e Subtração

######Diferença entre datas

In [0]:
orders_df = spark.read\
    .format("csv")\
    .option("header", "true")\
    .option("inferSchema", "true")\
    .load("/Volumes/workspace/default/arquivos-cursoudemy/Bike/orders.csv")
display(orders_df)

In [0]:
from pyspark.sql.functions import datediff, months_between, year

orders_df = orders_df.withColumn("Diferenca_Dias",datediff("required_date","order_date"))\
                    .withColumn("Diferenca_Meses",months_between("required_date","order_date"))\
                    .withColumn("Diferenca_Anos",year("required_date")-year("order_date"))

display(orders_df)

In [0]:
#Simulando diferença de alguma coluna de data com a data Atual 
from pyspark.sql.functions import datediff, months_between, year ,current_date

orders_df = orders_df.withColumn("diferenca_dias", datediff( current_date(),"required_date")) \
                     .withColumn("diferenca_meses", months_between(current_date(),"required_date")) \
                     .withColumn("diferenca_anos", year(current_date()) -  year("required_date"))

display(orders_df)

###### Adicionar intervalos em Datas

In [0]:
orders_df = spark.read\
    .format("csv")\
    .option("header", "true")\
    .option("inferSchema", "true")\
    .load("/Volumes/workspace/default/arquivos-cursoudemy/Bike/orders.csv")
display(orders_df)

In [0]:
from pyspark.sql.functions import expr

# Adicionar coluna 'previsao_entrega' com 5 dias adicionados à 'order_date'
# Adicionar coluna 'pagamento' com 1 mês adicionado à 'order_date'
# Adicionar coluna 'devolucao' com 1 Ano adicionado à 'order_date'

orders_df = orders_df\
                     .withColumn("previsao_entrega", expr("order_date + interval 5 days"))\
                     .withColumn("pagamento", expr("order_date + interval 1 month"))\
                     .withColumn("max_devolucao", expr("order_date + interval 1 year"))
display(orders_df)

In [0]:
#simulação rápida tirando intervalos
orders_df = orders_df\
                     .withColumn("previsao_entrega", expr("order_date - interval 5 days"))\
                     .withColumn("pagamento", expr("order_date - interval 1 month"))\
                     .withColumn("max_devolucao", expr("order_date - interval 1 year"))
display(orders_df)


##### Datas em SQL


In [0]:
orders_df = spark.read\
    .format("csv")\
    .option("header", "true")\
    .option("inferSchema", "true")\
    .load("/Volumes/workspace/default/arquivos-cursoudemy/Bike/orders.csv")
display(orders_df)

In [0]:
orders_df.createOrReplaceTempView("tabela_ordens")

In [0]:
%sql
select * from tabela_ordens

In [0]:
%sql
--extraindo partes de uma data
select*,
        YEAR(order_date) as Ano,
        MONTH(order_date) as Mes,
        DAY(order_date) as Dia
from tabela_ordens

In [0]:
%sql 
-- para treinar   
-- traduções: now ou  current_timestamp  = carimbo de data/hora atual
-- muito usado para ter coluna ou colunas de quando os daos foram carregados para base, arquivos banco de dados etc
--  converter fuso horário from_utc_timestamp

select * ,
now() as agora,
current_timestamp() AS agora2,
from_utc_timestamp(current_timestamp(), 'America/Sao_Paulo') AS agora_PTBR,
date_format(from_utc_timestamp(current_timestamp(), 'America/Sao_Paulo'), 'yyyy-MM-dd HH:mm:ss') AS agora_formatado,
date_format(from_utc_timestamp(current_timestamp(), 'America/Sao_Paulo'), 'dd/MM/yyyy') AS agora_formatado_brasil,
date_format(from_utc_timestamp(current_timestamp(), 'America/Sao_Paulo'), 'dd/MM/yyyy HH:mm:ss') AS agora_formatado_brasil_Completo,
year(from_utc_timestamp(current_timestamp(), 'America/Sao_Paulo')) as ano,
month(from_utc_timestamp(current_timestamp(), 'America/Sao_Paulo')) as mes,
day(from_utc_timestamp(current_timestamp(), 'America/Sao_Paulo')) as dia,
hour(from_utc_timestamp(current_timestamp(), 'America/Sao_Paulo')) as hora,
minute(from_utc_timestamp(current_timestamp(), 'America/Sao_Paulo')) as minuto,
second(from_utc_timestamp(current_timestamp(), 'America/Sao_Paulo')) as segundo
from tabela_ordens

In [0]:
%sql 
-- dados maior que certa data 
 SELECT * FROM tabela_ordens WHERE order_date >= '2016-01-01' -- Obs o padrao é ano-mes-dia


In [0]:
%sql 
-- quantos anos tenho de registro na base?
 SELECT distinct 
 year(order_date) 
 FROM tabela_ordens 

In [0]:
%sql 
--Dados entre 2017  e 2018
SELECT * FROM tabela_ordens
 WHERE year(order_date) BETWEEN 2017 and 2018

In [0]:
%sql 
--Dados entre datas espesificas 
SELECT * FROM tabela_ordens
WHERE order_date BETWEEN '2017-01-04' and '2018-01-01' 
 -- Obs o padrao é ano-mes-dia 

In [0]:
%sql
select * ,
date_format(order_date, 'yyyy-MM-dd HH:mm:ss') AS ptbr,
date_format(order_date, 'dd/MM/yyyy') AS dataformatada,
date_format(order_date, 'dd/MM/yyyy HH:mm:ss') AS datahora

from tabela_ordens

In [0]:
#Salvar consulta em um Df para trabalhar com Python ou salvar como qualqer tipo de arquivo 

resultado = spark.sql(
    
"""
select * ,
date_format(order_date, 'yyyy-MM-dd HH:mm:ss') AS ptbr,
date_format(order_date, 'dd/MM/yyyy') AS dataformatada,
date_format(order_date, 'dd/MM/yyyy HH:mm:ss') AS datahora

from tabela_ordens                



""")


display(resultado)
